In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, roc_curve
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import time




c:\Users\happy\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


ModuleNotFoundError: No module named 'torch'

In [90]:
df = pd.read_csv("./rdr2.csv")

df = df.dropna(subset=['review', 'labels'])

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['review'].tolist(), df['labels'].tolist(), test_size=0.2, random_state=42
)

print(train_texts)


In [91]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)
        

In [92]:
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)

In [93]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10
)

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    return {
        "accuracy": accuracy_score(labels, preds),
        "precision": precision_score(labels, preds),
        "recall": recall_score(labels, preds),
        "f1": f1_score(labels, preds),
        "roc_auc": roc_auc_score(labels, preds),
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)


start_time = time.time()
trainer.train()
training_time = time.time() - start_time
print(f"Eğitimin tamamlanması için geçen süre: {training_time:.2f} saniye")

""


In [94]:
results = trainer.evaluate()
print("Model Değerlendirme Sonuçları:", results)

' tokens = [word_tokenize(data) for data in review_list]\ndf_tokens = pd.DataFrame(tokens)\ndf_tokens.head(len(df_tokens)) '

In [95]:
from torch.utils.data import DataLoader

val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


print("\nÇıkarım (Inference) süresi hesaplanıyor...")

model.eval()

start_inference_time = time.time()

with torch.no_grad():
    for batch in val_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)

end_inference_time = time.time()

total_inference_time = end_inference_time - start_inference_time
num_samples = len(val_dataset)
average_inference_time = total_inference_time / num_samples

print(f"Toplam çıkarım süresi: {total_inference_time:.2f} saniye")
print(f"Veri başına ortalama çıkarım süresi: {average_inference_time:.6f} saniye")




' tagged_reviews = [nltk.pos_tag(review) for review in tokens] '

In [ ]:
predictions = trainer.predict(val_dataset)
preds = np.argmax(predictions.predictions, axis=1)
labels = predictions.label_ids
y_probs = predictions.predictions[:, 1]


cm = confusion_matrix(labels, preds)

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Negative", "Positive"],
            yticklabels=["Negative", "Positive"])

plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
fpr, tpr, _ = roc_curve(labels, y_probs)
plt.figure()
plt.plot(fpr, tpr, label="ROC curve (AUC = {:.2f})".format(roc_auc_score(labels, preds)))
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()
plt.show()

In [ ]:
tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()


accuracy = accuracy_score(labels, preds)
precision = precision_score(labels, preds)
recall = recall_score(labels, preds)
specificity = tn / (tn + fp)
f1 = f1_score(labels, preds)
roc_auc = roc_auc_score(labels, y_probs)

metrics_df = pd.DataFrame({
    "Metric": ["Accuracy", "Precision", "Recall (Sensitivity)", "Specificity", "F1-Score", "ROC-AUC"],
    "Score": [accuracy, precision, recall, specificity, f1, roc_auc]
})


print(metrics_df)

In [ ]:
train_losses = []
eval_losses = []
epochs = []

epoch_count = 0


for log in trainer.state.log_history:
    if "loss" in log:
        train_losses.append(log["loss"])
    if "eval_loss" in log:
        eval_losses.append(log["eval_loss"])
        epoch_count += 1
        epochs.append(epoch_count)

if len(train_losses) > len(eval_losses):
    train_losses = train_losses[:len(eval_losses)]
elif len(eval_losses) > len(train_losses):
    eval_losses = eval_losses[:len(train_losses)]

plt.figure(figsize=(10, 6))
plt.plot(epochs, train_losses, label="Training Loss", marker="o")
plt.plot(epochs, eval_losses, label="Validation Loss", marker="o")

plt.title("Epoch vs Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid()
plt.show()